![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `student_id`   | A unique ID for each student.                 |
| `city`  | A code for the city the student lives in.  |
| `city_development_index` | A scaled development index for the city.       |
| `gender` | The student's gender.       |
| `relevant_experience` | An indicator of the student's work relevant experience.       |
| `enrolled_university` | The type of university course enrolled in (if any).       |
| `education_level` | The student's education level.       |
| `major_discipline` | The educational discipline of the student.       |
| `experience` | The student's total work experience (in years).       |
| `company_size` | The number of employees at the student's current employer.       |
| `last_new_job` | The number of years between the student's current and previous jobs.       |
| `training_hours` | The number of hours of training completed.       |
| `job_change` | An indicator of whether the student is looking for a new job (`1`) or not (`0`).       |

In [93]:
# Start your code here!
import pandas as pd
import numpy as np

In [94]:
# Import the dataset
dfcustomers = pd.read_csv("customer_train.csv")
# Checking the data types
print(dfcustomers.dtypes)
display(dfcustomers.head())

student_id                  int64
city                       object
city_development_index    float64
gender                     object
relevant_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int64
job_change                  int64
dtype: object


student_id      city  ...  training_hours job_change
0        8949  city_103  ...              36          1
1       29725   city_40  ...              47          0
2       11561   city_21  ...              83          0
3       33241  city_115  ...              52          1
4         666  city_162  ...               8          0

[5 rows x 14 columns]

Once imported, let's filter the dataset to get only the students with at least 10 years of experience at companies with at least 1000 employees.

In [95]:
# Exploring the values in the column "experience"
print(dfcustomers["experience"].value_counts(dropna=False))
# Selecting only the values 10+ ensuring that the value "1" is not included
filter_experience = dfcustomers["experience"].str.contains("^1.|20")
# printing some values to check if the filter works correctly
print(list(zip(dfcustomers["experience"], filter_experience))[0:15])

>20    3286
5      1430
4      1403
3      1354
6      1216
2      1127
7      1028
10      985
9       980
8       802
15      686
11      664
14      586
1       549
<1      522
16      508
12      494
13      399
17      342
19      304
18      280
20      148
NaN      65
Name: experience, dtype: int64
[('>20', True), ('15', True), ('5', False), ('<1', False), ('>20', True), ('11', True), ('5', False), ('13', True), ('7', False), ('17', True), ('2', False), ('5', False), ('>20', True), ('2', False), ('5', False)]


In [96]:
# Exploring the values in the column "company_size"
print(dfcustomers["company_size"].value_counts(dropna=False))
# Selecting only the values 1000+ employees
filter_employees = dfcustomers["company_size"].str.contains("000")
# printing some values to check if the filter works correctly
print(list(zip(dfcustomers["company_size"], filter_employees))[0:15])

NaN          5938
50-99        3083
100-499      2571
10000+       2019
10-49        1471
1000-4999    1328
<10          1308
500-999       877
5000-9999     563
Name: company_size, dtype: int64
[(nan, nan), ('50-99', False), (nan, nan), (nan, nan), ('50-99', False), (nan, nan), ('50-99', False), ('<10', False), ('50-99', False), ('10000+', True), (nan, nan), ('5000-9999', True), ('1000-4999', True), (nan, nan), (nan, nan)]


In [97]:
# Filtering the dataset considering the two conditions
ds_jobs = dfcustomers[filter_experience & filter_employees]
# Checking dimensions
print(dfcustomers.shape)
print(ds_jobs.shape)

(19158, 14)
(2201, 14)


Once filtered the dataset, I'll select the different target data types to convert them into the desired data types.

In [98]:
# Converting the integers into int32
col_integers = ds_jobs.select_dtypes("int64").astype("int32")
print(col_integers.dtypes)

student_id        int32
training_hours    int32
job_change        int32
dtype: object


In [99]:
# Converting the float-typed columns into float16
col_float = ds_jobs.select_dtypes("float64").astype("float16")
print(col_float.dtypes)

city_development_index    float16
dtype: object


For the object-typed columns, I'll check first for the unique values and the values frequency.

In [100]:
# Checking the unique values for each object-typed column
display(ds_jobs.select_dtypes("object").describe())

city gender  ... company_type last_new_job
count       2201   1821  ...         2144         2184
unique        82      3  ...            4            6
top     city_103   Male  ...      Pvt Ltd           >4
freq         805   1654  ...         1807          828

[4 rows x 10 columns]

In [101]:
# Converting all object columns into category
col_objects = ds_jobs.select_dtypes("object").astype("category")
print(col_objects.dtypes)

city                   category
gender                 category
relevant_experience    category
enrolled_university    category
education_level        category
major_discipline       category
experience             category
company_size           category
company_type           category
last_new_job           category
dtype: object


In [102]:
# Checking for the categories in each column
for i in col_objects.columns:
    print(col_objects[i].cat.categories)

Index(['city_1', 'city_10', 'city_100', 'city_102', 'city_103', 'city_104',
       'city_105', 'city_106', 'city_107', 'city_11', 'city_114', 'city_115',
       'city_116', 'city_118', 'city_12', 'city_123', 'city_126', 'city_13',
       'city_131', 'city_133', 'city_134', 'city_136', 'city_138', 'city_14',
       'city_141', 'city_142', 'city_143', 'city_144', 'city_145', 'city_149',
       'city_150', 'city_152', 'city_157', 'city_158', 'city_159', 'city_16',
       'city_160', 'city_162', 'city_165', 'city_167', 'city_173', 'city_175',
       'city_19', 'city_20', 'city_21', 'city_23', 'city_24', 'city_27',
       'city_28', 'city_36', 'city_37', 'city_39', 'city_40', 'city_41',
       'city_45', 'city_46', 'city_50', 'city_53', 'city_57', 'city_59',
       'city_61', 'city_64', 'city_65', 'city_67', 'city_69', 'city_7',
       'city_71', 'city_72', 'city_73', 'city_75', 'city_76', 'city_77',
       'city_80', 'city_83', 'city_89', 'city_90', 'city_91', 'city_93',
       'city_94', 

Now, we'll select the variables that required to be set as ordinal such as `relevant_experience`, `enrolled_university`, `education_level`, `experience`, `company_size` and `last_new_job`.

In [103]:
# Reordering the categories and setting them as ordinal
col_objects["relevant_experience"].cat.reorder_categories(new_categories = ['No relevant experience', 'Has relevant experience'], ordered = True, inplace = True)
col_objects["enrolled_university"].cat.reorder_categories(new_categories = ['no_enrollment', 'Part time course', 'Full time course'], ordered = True, inplace = True)
col_objects["education_level"].cat.reorder_categories(new_categories = ['Primary School', 'High School', 'Graduate', 'Masters', 'Phd'], ordered = True, inplace = True)
col_objects["experience"].cat.reorder_categories(new_categories = ['10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '>20'], ordered = True, inplace = True)
col_objects["company_size"].cat.reorder_categories(new_categories = ['1000-4999', '5000-9999', '10000+'], ordered = True, inplace = True)
col_objects["last_new_job"].cat.reorder_categories(new_categories = ['never', '1', '2', '3', '4', '>4'], ordered = True, inplace = True)

# Checking for the categories in each column again
for i in col_objects.columns:
    print(col_objects[i].cat.categories)

Index(['city_1', 'city_10', 'city_100', 'city_102', 'city_103', 'city_104',
       'city_105', 'city_106', 'city_107', 'city_11', 'city_114', 'city_115',
       'city_116', 'city_118', 'city_12', 'city_123', 'city_126', 'city_13',
       'city_131', 'city_133', 'city_134', 'city_136', 'city_138', 'city_14',
       'city_141', 'city_142', 'city_143', 'city_144', 'city_145', 'city_149',
       'city_150', 'city_152', 'city_157', 'city_158', 'city_159', 'city_16',
       'city_160', 'city_162', 'city_165', 'city_167', 'city_173', 'city_175',
       'city_19', 'city_20', 'city_21', 'city_23', 'city_24', 'city_27',
       'city_28', 'city_36', 'city_37', 'city_39', 'city_40', 'city_41',
       'city_45', 'city_46', 'city_50', 'city_53', 'city_57', 'city_59',
       'city_61', 'city_64', 'city_65', 'city_67', 'city_69', 'city_7',
       'city_71', 'city_72', 'city_73', 'city_75', 'city_76', 'city_77',
       'city_80', 'city_83', 'city_89', 'city_90', 'city_91', 'city_93',
       'city_94', 

Now let's update the dataset with the new data types.

In [104]:
# Concatenating the columns with the desired data types
ds_jobs_clean = pd.concat([col_integers, col_float, col_objects], axis = 1)
# Reordering the columns as the original dataset
ds_jobs_clean = ds_jobs_clean.reindex(ds_jobs.columns, axis = 1)
print(ds_jobs_clean.shape)
print(ds_jobs_clean.dtypes)

(2201, 14)
student_id                   int32
city                      category
city_development_index     float16
gender                    category
relevant_experience       category
enrolled_university       category
education_level           category
major_discipline          category
experience                category
company_size              category
company_type              category
last_new_job              category
training_hours               int32
job_change                   int32
dtype: object


In [105]:
# Comparing efficiency of cleaning the dataset
# Raw dataset
print(ds_jobs.info())
print(ds_jobs.memory_usage())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2201 entries, 9 to 19143
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              2201 non-null   int64  
 1   city                    2201 non-null   object 
 2   city_development_index  2201 non-null   float64
 3   gender                  1821 non-null   object 
 4   relevant_experience     2201 non-null   object 
 5   enrolled_university     2185 non-null   object 
 6   education_level         2184 non-null   object 
 7   major_discipline        2097 non-null   object 
 8   experience              2201 non-null   object 
 9   company_size            2201 non-null   object 
 10  company_type            2144 non-null   object 
 11  last_new_job            2184 non-null   object 
 12  training_hours          2201 non-null   int64  
 13  job_change              2201 non-null   int64  
dtypes: float64(1), int64(3), object(10)
mem

In [106]:
# Clean dataset
print(ds_jobs_clean.info())
print(ds_jobs_clean.memory_usage())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2201 entries, 9 to 19143
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              2201 non-null   int32   
 1   city                    2201 non-null   category
 2   city_development_index  2201 non-null   float16 
 3   gender                  1821 non-null   category
 4   relevant_experience     2201 non-null   category
 5   enrolled_university     2185 non-null   category
 6   education_level         2184 non-null   category
 7   major_discipline        2097 non-null   category
 8   experience              2201 non-null   category
 9   company_size            2201 non-null   category
 10  company_type            2144 non-null   category
 11  last_new_job            2184 non-null   category
 12  training_hours          2201 non-null   int32   
 13  job_change              2201 non-null   int32   
dtypes: category(10), float1